In [9]:
from pseudo_povm_circuits import *
import cmath
import math

In [10]:
def generate_random_pseudo_povm():
  '''Generates random pseudo POVM

  Returns instance of PseudoPovm class
  '''
  
  povm_ten = tf.random.normal([4, 3], dtype = tf.float64)
  return PseudoPovm(povm_ten)

In [11]:
def total_negativity_optimization(circuit):
  '''Finds pseudo POVM where total negativity of given circuit is minimal

  Arguments
  ---------
  circuit : QubitCircuit 

  Returns instance of class PseudoPovm
  '''


  #=========================#
  lr = 0.02 # learning rate
  #=========================#
  opt = tf.keras.optimizers.Adam(lr)

  def neg(var):
    povm = PseudoPovm(var)
    return circuit.total_negativity(povm) 

  povm = generate_random_pseudo_povm()


  var = povm.lam
  var = tf.Variable(var)


  #print(neg(var))

  for i in range(1000):
    with tf.GradientTape() as tape:

      loss = neg(var)
    grad = tape.gradient(loss, var) 
    opt.apply_gradients(zip([grad], [var]))  # minimization step
  
  lam = tf.complex(var,tf.zeros([4, 3], dtype = tf.float64))
  return PseudoPovm(lam)

In [12]:
# Initialize some single qubit gates
X = sigma_x
Y = sigma_y
Z = sigma_z
T = tf.constant([[1 ,0],[0, cmath.exp(1j * math.pi / 4)]], dtype=tf.complex128)
H = tf.constant([[1,1],[1,-1]], dtype=tf.complex128) / math.sqrt(2)
S = tf.constant([[1,0],[0,1j]],dtype=tf.complex128)

In [13]:
# Create pseudo POVM
povm = generate_random_pseudo_povm()

In [14]:
povm.get_pseudo_povm_in_comp_basis()

<tf.Tensor: shape=(4, 2, 2), dtype=complex128, numpy=
array([[[ 1.05580108e+00+0.j        , -1.51188362e+00+0.07748226j],
        [-1.51188362e+00-0.07748226j, -1.35561061e+00+0.j        ]],

       [[ 7.75041129e-01+0.j        , -5.11341818e-01+0.00522882j],
        [-5.11341818e-01-0.00522882j,  1.16968443e+00+0.j        ]],

       [[ 3.52813960e-01+0.j        ,  2.02364469e+00+0.0701305j ],
        [ 2.02364469e+00-0.0701305j ,  9.93286761e-01+0.j        ]],

       [[-1.18365617e+00+0.j        , -4.19255546e-04-0.15284157j],
        [-4.19255546e-04+0.15284157j,  1.92639420e-01+0.j        ]]])>

In [15]:
# Create quantum circuit
circuit = QubitCircuit([X,Y,H])

In [16]:
# Run circuit 
circuit.run_circuit(povm)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.5, 0.5])>

In [17]:
# Check total negativity before optimization
circuit.total_negativity(povm)

<tf.Tensor: shape=(), dtype=float64, numpy=10.580866833241403>

In [18]:
# Find pseudo POVM where negativity is minimal
optimal_basis = total_negativity_optimization(circuit)

In [19]:
# Let's check total negativity in nev pseudo POVM
circuit.total_negativity(optimal_basis)

<tf.Tensor: shape=(), dtype=float64, numpy=2.1292528167329907>